In [1]:
%load_ext lab_black

In [2]:
import warnings

import numpy as np
import pandas as pd

from scipy.fft import fft
from statsmodels.tsa.api import ExponentialSmoothing

In [3]:
data = pd.read_csv("../data/combined.csv").set_index("date")

# Advanced Features

In addition to the basic features which should likely always be calculated for time series, there are some more specific features which I will calculate in this notebook. These are:

* Fourier frequencies
* Price momentum
* Inflation-price quadratic features
* Smoothed data

First, the Fourier components. How they *should* have been calculated is by writing a simple windowing function, taking the FFT, and rehshaping the output into columns to be aggregated in a dataframe. The way that *I* calculated it is by using pandas windowing functions. However, pandas windows do not allow you to output arrays. Therefore, I wrote a hacky solution which applies an aggregator function that returns a dummy variable while appending to a global-scope list.

In [4]:
# fft with domain size of 52

FFT_DOMAIN = 52
# frequencies of interest
FFT_FREQS = [0, 1, 3, 7, 12, 25, 51]

fft_accumulator = []


def hack(x):
    # this is a crime
    fft_accumulator.append(fft(x.values, norm="ortho"))

    return 0

# dummy df to store index and dummy output
dummy_fourier = data.rolling(window=FFT_DOMAIN).apply(hack)

num_cols = len(data.columns)
num_freq_blocks = len(fft_accumulator)
blocks_per_column = num_freq_blocks // num_cols

fft_accumulator = np.array(fft_accumulator)

dfs = []
for n, col in enumerate(data.columns):
    # frequencies corresponding to the column are selected
    freq_block = fft_accumulator[n * blocks_per_column : (n + 1) * blocks_per_column][
        :, FFT_FREQS
    ]
    # all of the relevant frequencies are appended
    dfs.append(pd.DataFrame(freq_block, columns=[f"{col}_f{m}" for m in FFT_FREQS]))

fourier = pd.concat(dfs, axis=1, join="inner")
fourier["date"] = data.index[51:]
fourier = fourier.set_index("date")

# real and imaginary components are split to ensure real valued features after PCA
real_imag_split = (
    {f"{col}_real": fourier[col].values.real for col in fourier}
    | {f"{col}_imag": fourier[col].values.imag for col in fourier}
    | {"date": fourier.index}
)
fourier = pd.DataFrame(real_imag_split).set_index("date")
fourier

,inflation_one_month_f0_real,inflation_one_month_f1_real,inflation_one_month_f3_real,inflation_one_month_f7_real,inflation_one_month_f12_real,inflation_one_month_f25_real,inflation_one_month_f51_real,inflation_two_month_f0_real,inflation_two_month_f1_real,inflation_two_month_f3_real,...,brent_price_f12_imag,brent_price_f25_imag,brent_price_f51_imag,gas_price_f0_imag,gas_price_f1_imag,gas_price_f3_imag,gas_price_f7_imag,gas_price_f12_imag,gas_price_f25_imag,gas_price_f51_imag
date,,,,,,,,,,,,,,,,,,,,,
1994-03-28,1.830511,0.007644,0.048824,-0.106198,0.018646,0.016701,0.007644,3.536214,0.118925,0.386436,...,-0.671046,0.046335,6.989783,-0.0,-0.133535,0.023961,-0.000181,-0.006389,-0.000552,0.133535
1994-04-04,1.844378,-0.014780,-0.045971,-0.156216,0.031289,-0.031502,-0.014780,3.536214,0.048364,0.255614,...,-0.465494,-0.105098,6.961508,-0.0,-0.138389,0.017112,0.004847,-0.007637,0.000686,0.138389
1994-04-11,1.830511,-0.064623,-0.161626,-0.124046,-0.026642,0.045742,-0.064623,3.536214,-0.022902,0.091569,...,-0.174921,0.073236,6.920812,-0.0,-0.142179,0.005237,0.000692,-0.003299,-0.001763,0.142179
1994-04-18,1.816643,-0.113422,-0.255374,-0.003628,-0.025516,-0.031682,-0.113422,3.536214,-0.093835,-0.084376,...,-0.178266,-0.113352,6.852241,-0.0,-0.144748,-0.009828,-0.009223,-0.000179,0.001962,0.144748
1994-04-25,1.802776,-0.160466,-0.315030,0.123906,0.032687,0.044794,-0.160466,3.536214,-0.163399,-0.249355,...,-0.371904,0.090636,6.744928,-0.0,-0.145975,-0.025877,-0.017699,-0.003077,-0.002901,0.145975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,3.244996,-0.249827,0.129240,-0.041313,0.001015,-0.020767,-0.249827,6.878282,-0.550819,-0.392713,...,-0.407363,-1.546817,25.868915,-0.0,-0.444636,0.181286,-0.013669,-0.020546,0.013401,0.444636
2023-01-02,3.217261,-0.213834,0.290896,0.039214,-0.011927,0.041003,-0.213834,6.808945,-0.488221,-0.207464,...,-1.513367,1.708773,32.009923,-0.0,-0.710343,-0.120580,-0.162430,-0.133055,-0.012552,0.710343
2023-01-09,3.147924,-0.215821,0.377648,0.075076,0.015487,0.035926,-0.215821,6.698005,-0.459297,-0.029644,...,0.547757,-1.784466,37.622809,-0.0,-0.966528,-0.409232,-0.206943,-0.018413,0.010684,0.966528


Sometimes, "momentum" is a quantity of interest for understanding commodity and stock prices, and it is defined as the current price divided by the previous price. Momentum effectively acts as a measure of stability, with quantity having momentum one being unlikely to change. Momentum may or may not be usefule to a model, however it is a nonlinear relationship between two features and thus cannot be easily learned by a neural network.

In [5]:
# momentum analogous quantity
PRICE_COLS = ["wti_price", "brent_price", "opec_price", "gas_price"]
MOMENTUM_SPREADS = [1, 4, 13, 26, 52]

momentum = {}

for col in PRICE_COLS:
    for spread in MOMENTUM_SPREADS:
        momentum[f"{col}_p{spread}"] = data[col] / data[col].shift(spread)

momentum = pd.DataFrame(momentum).fillna(1).replace([np.inf], 1)
momentum

,wti_price_p1,wti_price_p4,wti_price_p13,wti_price_p26,wti_price_p52,brent_price_p1,brent_price_p4,brent_price_p13,brent_price_p26,brent_price_p52,opec_price_p1,opec_price_p4,opec_price_p13,opec_price_p26,opec_price_p52,gas_price_p1,gas_price_p4,gas_price_p13,gas_price_p26,gas_price_p52
date,,,,,,,,,,,,,,,,,,,,
1993-04-05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1993-04-12,1.022000,1.000000,1.000000,1.000000,1.000000,1.016830,1.000000,1.000000,1.000000,1.000000,1.019521,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1993-04-19,0.996575,1.000000,1.000000,1.000000,1.000000,0.998932,1.000000,1.000000,1.000000,1.000000,0.997702,1.000000,1.000000,1.000000,1.000000,1.010300,1.000000,1.000000,1.000000,1.000000
1993-04-26,0.995582,1.000000,1.000000,1.000000,1.000000,0.997862,1.000000,1.000000,1.000000,1.000000,0.996673,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1993-05-03,0.983728,0.997500,1.000000,1.000000,1.000000,0.994108,1.007600,1.000000,1.000000,1.000000,0.988703,1.002343,1.000000,1.000000,1.000000,1.006487,1.016854,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,1.028196,0.894073,0.861073,0.645028,1.055353,1.028513,0.877403,0.883930,0.645987,1.095445,0.997319,0.906381,0.809218,0.673709,1.082433,0.964509,0.859421,0.857597,0.638373,0.952577
2023-01-02,1.028754,0.981458,0.925953,0.716352,1.078878,0.998515,0.933796,0.911227,0.690576,1.091598,1.052477,0.985735,0.915600,0.699532,1.062540,0.990414,0.877775,0.835856,0.643302,0.949037
2023-01-09,1.025233,0.999243,0.989386,0.714041,1.041815,1.014254,0.962366,0.937557,0.679990,1.046955,0.997811,1.030003,0.890493,0.718225,1.041508,1.039963,0.950628,0.852136,0.682722,0.985211


On a similar note of non-linear relationships, inflation as a number isn't super useful on its own. But, when multiplied on a price, it represents how much the price has increased lately as a result of economic factors. Therefore, we multiply all of the inflation columns by all of the price columns.

In [6]:
# inflation quadratics
inflation_contribution = {}
INFLATION_COLS = [col for col in data.columns if "inflation" in col]

for col in PRICE_COLS:
    for inflation in INFLATION_COLS:
        inflation_contribution[f"{col}_{inflation}_contribution"] = (
            data[col] * data[inflation] / 100.0
        )

inflation_contribution = pd.DataFrame(inflation_contribution)
inflation_contribution

,wti_price_inflation_one_month_contribution,wti_price_inflation_two_month_contribution,wti_price_inflation_three_month_contribution,wti_price_inflation_six_month_contribution,wti_price_inflation_twelve_month_contribution,brent_price_inflation_one_month_contribution,brent_price_inflation_two_month_contribution,brent_price_inflation_three_month_contribution,brent_price_inflation_six_month_contribution,brent_price_inflation_twelve_month_contribution,opec_price_inflation_one_month_contribution,opec_price_inflation_two_month_contribution,opec_price_inflation_three_month_contribution,opec_price_inflation_six_month_contribution,opec_price_inflation_twelve_month_contribution,gas_price_inflation_one_month_contribution,gas_price_inflation_two_month_contribution,gas_price_inflation_three_month_contribution,gas_price_inflation_six_month_contribution,gas_price_inflation_twelve_month_contribution
date,,,,,,,,,,,,,,,,,,,,
1993-04-05,0.08000,0.20000,0.30000,0.44000,0.68000,0.07368,0.18420,0.27630,0.40524,0.62628,0.07684,0.19210,0.28815,0.42262,0.653140,0.004272,0.010680,0.016020,0.023496,0.036312
1993-04-12,0.04088,0.12264,0.24528,0.36792,0.71540,0.03746,0.11238,0.22476,0.33714,0.65555,0.03917,0.11751,0.23502,0.35253,0.685475,0.002136,0.006408,0.012816,0.019224,0.037380
1993-04-19,0.04074,0.12222,0.24444,0.36666,0.71295,0.03742,0.11226,0.22452,0.33678,0.65485,0.03908,0.11724,0.23448,0.35172,0.683900,0.002158,0.006474,0.012948,0.019422,0.037765
1993-04-26,0.04056,0.12168,0.24336,0.36504,0.70980,0.03734,0.11202,0.22404,0.33606,0.65345,0.03895,0.11685,0.23370,0.35055,0.681625,0.002158,0.006474,0.012948,0.019422,0.037765
1993-05-03,0.03990,0.11970,0.23940,0.35910,0.69825,0.03712,0.11136,0.22272,0.33408,0.64960,0.03851,0.11553,0.23106,0.34659,0.673925,0.002172,0.006516,0.013032,0.019548,0.038010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.15024,0.22536,0.45072,1.35216,4.28184,0.16160,0.24240,0.48480,1.45440,4.60560,0.15626,0.23439,0.46878,1.40634,4.453410,0.006468,0.009702,0.019404,0.058212,0.184338
2023-01-02,0.15456,0.23184,0.46368,1.39104,4.40496,0.16136,0.24204,0.48408,1.45224,4.59876,0.16446,0.24669,0.49338,1.48014,4.687110,0.006406,0.009609,0.019218,0.057654,0.182571
2023-01-09,0.15846,0.23769,0.47538,1.42614,4.51611,0.16366,0.24549,0.49098,1.47294,4.66431,0.16410,0.24615,0.49230,1.47690,4.676850,0.006662,0.009993,0.019986,0.059958,0.189867


Finally, one of the struggles of time series is that the data is jagged and hard to predict. Trends and seasonality get muddied by noise, and it makes learning hard as the risk of overfitting is high. Thus, it makes sense to create a *smoothed* version of the data, which captures the general trends of the data without each individual spike being represented. Triple Exponential (Holt-Winters) smoothing is the go-to technique for time series data because it does not infer from the future, and because it accounts for stationarity, seasonality, and trend. TES itself can actually be used for forecasting, but we will just use it for smoothing.

In [7]:
with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")

    smoothed = {}
    for col in data.columns:
        fit = ExponentialSmoothing(
            data[col].values,
            seasonal_periods=52,
            trend="add",
            seasonal="add",
            damped_trend=True,
            initialization_method="estimated",
        ).fit()

        smoothed[col] = fit.fittedvalues

smoothed = pd.DataFrame({f"{col}_smoothed": val for col, val in smoothed.items()})
smoothed["date"] = data.index
smoothed = smoothed.set_index("date")
smoothed

,inflation_one_month_smoothed,inflation_two_month_smoothed,inflation_three_month_smoothed,inflation_six_month_smoothed,inflation_twelve_month_smoothed,oil_reserves_algeria_smoothed,oil_reserves_kuwait_smoothed,oil_reserves_united_states_smoothed,oil_reserves_middle_east_bp_smoothed,oil_reserves_iran_smoothed,oil_reserves_vietnam_smoothed,oil_reserves_lower_middle_income_countries_smoothed,oil_reserves_nigeria_smoothed,oil_reserves_libya_smoothed,oil_reserves_congo_smoothed,opec_price_smoothed,wti_price_smoothed,brent_price_smoothed,gas_price_smoothed
date,,,,,,,,,,,,,,,,,,,
1993-04-05,0.399805,0.999995,1.500065,2.195912,3.410895,1.255572e+09,1.313299e+10,4.256604e+09,9.023799e+10,1.266089e+10,3.998190e+07,1.935439e+10,2.925281e+09,3.164809e+09,9.272948e+07,19.300987,19.999874,18.419867,1.067953
1993-04-12,0.166695,0.673973,1.313217,1.854518,3.423323,1.254882e+09,1.314546e+10,4.325955e+09,9.017818e+10,1.266669e+10,3.435525e+07,1.928150e+10,2.829907e+09,3.110827e+09,9.353651e+07,19.036926,20.065987,18.686708,1.066945
1993-04-19,0.207951,0.667053,1.311436,1.765959,3.500402,1.254879e+09,1.314414e+10,4.321887e+09,9.018102e+10,1.266675e+10,3.442476e+07,1.928842e+10,2.833368e+09,3.106655e+09,9.339294e+07,19.880469,20.381401,18.694441,1.079419
1993-04-26,0.225578,0.678907,1.318143,1.774716,3.514181,1.254879e+09,1.314276e+10,4.317453e+09,9.018418e+10,1.266679e+10,3.452278e+07,1.929573e+10,2.840817e+09,3.106256e+09,9.324463e+07,18.930755,20.668959,18.516822,1.092277
1993-05-03,0.225126,0.658577,1.321113,1.799035,3.509196,1.254879e+09,1.314137e+10,4.313016e+09,9.018733e+10,1.266682e+10,3.462099e+07,1.930304e+10,2.848427e+09,3.106141e+09,9.309631e+07,20.399237,19.897579,18.306847,1.074038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.186653,0.266503,0.531090,1.895258,5.668450,1.664078e+09,1.384441e+10,9.377904e+09,1.140150e+11,2.152425e+10,6.001268e+08,3.177584e+10,5.018295e+09,6.627938e+09,3.931019e+08,78.048971,73.223194,77.704994,3.266742
2023-01-02,0.316207,0.381942,0.578055,1.946936,5.670663,1.664079e+09,1.384440e+10,9.377811e+09,1.140137e+11,2.152405e+10,6.001290e+08,3.177536e+10,5.036163e+09,6.616677e+09,3.931060e+08,77.447456,74.942097,80.574578,3.145804
2023-01-09,0.621580,0.610980,0.728839,2.057272,5.686803,1.664079e+09,1.384439e+10,9.377528e+09,1.140138e+11,2.152425e+10,6.001262e+08,3.177514e+10,5.032281e+09,6.613993e+09,3.931055e+08,81.268532,76.812742,80.107295,3.156236


In [8]:
data_advanced = pd.concat(
    [fourier, momentum, inflation_contribution, smoothed], join="inner", axis=1
)
data_advanced

,inflation_one_month_f0_real,inflation_one_month_f1_real,inflation_one_month_f3_real,inflation_one_month_f7_real,inflation_one_month_f12_real,inflation_one_month_f25_real,inflation_one_month_f51_real,inflation_two_month_f0_real,inflation_two_month_f1_real,inflation_two_month_f3_real,...,oil_reserves_iran_smoothed,oil_reserves_vietnam_smoothed,oil_reserves_lower_middle_income_countries_smoothed,oil_reserves_nigeria_smoothed,oil_reserves_libya_smoothed,oil_reserves_congo_smoothed,opec_price_smoothed,wti_price_smoothed,brent_price_smoothed,gas_price_smoothed
date,,,,,,,,,,,,,,,,,,,,,
1994-03-28,1.830511,0.007644,0.048824,-0.106198,0.018646,0.016701,0.007644,3.536214,0.118925,0.386436,...,1.266638e+10,5.090679e+07,1.926405e+10,2.892164e+09,3.106593e+09,9.970379e+07,14.581898,15.497525,14.406717,1.026407
1994-04-04,1.844378,-0.014780,-0.045971,-0.156216,0.031289,-0.031502,-0.014780,3.536214,0.048364,0.255614,...,1.266639e+10,5.119078e+07,1.926528e+10,2.890678e+09,3.102347e+09,9.977294e+07,14.760769,15.408856,14.776587,1.020572
1994-04-11,1.830511,-0.064623,-0.161626,-0.124046,-0.026642,0.045742,-0.064623,3.536214,-0.022902,0.091569,...,1.266639e+10,5.147656e+07,1.926652e+10,2.901407e+09,3.110185e+09,9.984188e+07,14.491533,15.106556,14.617616,1.013435
1994-04-18,1.816643,-0.113422,-0.255374,-0.003628,-0.025516,-0.031682,-0.113422,3.536214,-0.093835,-0.084376,...,1.266640e+10,5.176063e+07,1.926775e+10,2.900372e+09,3.106922e+09,9.991107e+07,14.086759,14.202841,13.015352,1.022682
1994-04-25,1.802776,-0.160466,-0.315030,0.123906,0.032687,0.044794,-0.160466,3.536214,-0.163399,-0.249355,...,1.266641e+10,5.204528e+07,1.926898e+10,2.902690e+09,3.106306e+09,9.998018e+07,14.150259,15.933686,14.031396,1.044330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,3.244996,-0.249827,0.129240,-0.041313,0.001015,-0.020767,-0.249827,6.878282,-0.550819,-0.392713,...,2.152425e+10,6.001268e+08,3.177584e+10,5.018295e+09,6.627938e+09,3.931019e+08,78.048971,73.223194,77.704994,3.266742
2023-01-02,3.217261,-0.213834,0.290896,0.039214,-0.011927,0.041003,-0.213834,6.808945,-0.488221,-0.207464,...,2.152405e+10,6.001290e+08,3.177536e+10,5.036163e+09,6.616677e+09,3.931060e+08,77.447456,74.942097,80.574578,3.145804
2023-01-09,3.147924,-0.215821,0.377648,0.075076,0.015487,0.035926,-0.215821,6.698005,-0.459297,-0.029644,...,2.152425e+10,6.001262e+08,3.177514e+10,5.032281e+09,6.613993e+09,3.931055e+08,81.268532,76.812742,80.107295,3.156236


In [9]:
data_advanced.to_csv("../data/advanced_features.csv")